# Application to CTA Data


Notebook stolen and modified from https://github.com/tudo-astroparticlephysics/machine-learning-lecture/blob/main/smd_boosting.ipynb

main author: Maximilian Nöthe

additional authors: Thomas Vuillaume, Michaël Dell'aiera, Martino Sorbaro

In [ ]:
import ml_func as ml

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.colors import ListedColormap

In [ ]:
pd.options.display.max_rows = 10
ml.set_plot_style()

# A Complete Example

Below we load a dataset containing data from simulated CTA Observations.

<img width="100%" src="https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/CTA_Telescopes_in_Southern_Hemisphere.jpg/1280px-CTA_Telescopes_in_Southern_Hemisphere.jpg" />   

We will perform the typical steps to build and evaluate a classifier.

0. Understand where your data comes from

1. Preprocessing
    * Drop Constant Values,
    * Handle Missing Data 
    * Feature Generation

2. Splitting
    
    * Split your data into training and evaluation sets
    
3. Training 
    
    * Train your classifier of choice.
    
4. Evaluation
    
    * Evaluate the performance on the test data set.
    * If not good enough, go back to step 1 
    
5. Physics
    
    * Check whether your data support your hypothesis
    

## 1. Get to know your data

Cherenkov telescopes record short flashes of light produced by very high energy cosmic rays and photons hitting earths atmosphere.

![](https://www.cta-observatory.org/wp-content/uploads/2016/05/cta47.png)

In [ ]:
%%HTML
<!-- https://nextcloud.e5.physik.tu-dortmund.de/index.php/s/e7yb2mifGDeyDBN/download -->
<video width="100%" controls>
  <source src="./resources/fact_events.mp4" type="video/mp4">
</video>

We will use machine learning for two tasks in this example. 

 * Train a classifier to distinguish events induced by gamma rays form events induced by cosmic rays
 * Train a regressor to estimate the energy of the incoming primary particle.

In [ ]:
# We can visualize the array of telescopes

array = ml.SubarrayDescription.from_hdf(ml.gamma_filename)
array.peek()

## 2. Preprocess data

A lot of preprocessing has already happened at this point.

* Calibration of Raw Data
* Data Reduction from voltage timeseries per pixel to number of photons and mean time for each pixel
* Calculation of image features
* Geometrical Reconstruction of the Shower Geometry


Load data and remove unwanted columns and store the true labels separately.

In [ ]:
gammas = ml.get_gammas()

In [ ]:
len(gammas.columns)

Check the data types of the columns. We can select non-numeric types and encode them. But in this case we might as well drop them as the attribute is not important.

In [ ]:
c = gammas.select_dtypes(exclude=['number']).columns
print('Removed columns:', c.values)

gammas = gammas.drop(c, axis='columns')

We can spot the columns with constant values by looking at the standard deviation.

In [ ]:
desc = gammas.describe()
desc

In [ ]:
c = desc.columns[desc.loc['std'] == 0]
print('Removed columns:', c.values)
gammas = gammas.drop(c, axis='columns')

drop columns where all rows are nan

In [ ]:
c = gammas.columns[gammas.count() == 0]
print('Removed columns:', c.values)
gammas = gammas.drop(c, axis='columns')

Check for missing data. (Just delete it in this case)

In [ ]:
print(len(gammas))
gammas = gammas.dropna()
print(len(gammas))

So far we only loaded simulated gamma-ray showers. Now we do the same for the cosmic ray events. We create a method to perform all preprocessing in one step. We need this several times.

In [ ]:
def preprocess(df):
    
    c = df.select_dtypes(exclude=['number']).columns
    df = df.drop(c, axis='columns')
    
    c = df.columns[df.count() == 0]
    df = df.drop(c, axis='columns')
    
    desc = df.describe()
    
    c = desc.columns[desc.loc['std'] == 0]
    df = df.drop(c, axis='columns')
    
    df = df.dropna()
    
    return df

In [ ]:
protons = ml.get_protons()
protons = preprocess(protons)

Now we can perform feature generation. We use our expert knowledge or intuition to create a new feature by combining existing columns into a new variable.

In [ ]:
def feature_generation(df):
    df['awesome_feature'] =  df['hillas_intensity'] * (df['hillas_width'] / df['hillas_length'])
    
    # distance of impact point to the telescope
    df['impact'] = np.sqrt(
        (df['HillasReconstructor_core_x'] - df['pos_x'])**2
        + (df['HillasReconstructor_core_y'] - df['pos_y'])**2
    )
    return df

gammas = feature_generation(gammas)
protons = feature_generation(protons)

gammas[['awesome_feature', 'impact']]

## 3. A quick look at the data

In [ ]:
bins = np.linspace(-20, 20, 100)
# bins = np.logspace(0, 1, 100)
# bins = 100
bins = np.arange(0, 10) - 0.5
bins = np.geomspace(1e3, 30e3, 51)

col = 'HillasReconstructor_h_max'

plt.figure()
plt.hist(gammas[col], bins=bins, histtype='step', lw=2, label='Gammas', density=True)
plt.hist(protons[col], bins=bins, histtype='step', lw=2, label='Protons', density=True)
plt.xscale('log')
plt.xlabel(col)
plt.legend()
None

### Exercise
You may plot other histogram to get a better grasp at the data

In [ ]:
plt.figure(figsize=(15,10))
plt.scatter(gammas['true_core_x'], gammas['true_core_y'], s=1, alpha=0.4)
plt.axis('equal')

## 4. Train a regressor to reconstruct the gammas energy

In [ ]:
# we select the training features by removing columns with `true_` values coming from the simulation
training_features = [name for name in list(gammas.columns) if not name.startswith('true_')]
predict_feature = 'true_energy'

training_features

In [ ]:
from sklearn.model_selection import train_test_split

X = gammas[training_features]
y = gammas[predict_feature]

X_test, X_train, y_test, y_train = train_test_split(X, y)


In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
reg = DecisionTreeRegressor(max_depth=30)
reg.fit(X_train, y_train)

In [ ]:
y_predict = reg.predict(X_test)

In [ ]:
plt.figure()
plt.hist2d(y_test, y_predict, bins=np.logspace(-2, 2, 60))
plt.xscale('log')
plt.yscale('log')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_log_error, mean_squared_error

In [ ]:
mean_squared_log_error(y_test, y_predict)

Extra: How can we improve that regression?

## 5. Train a classifier to do gamma/hadron separation

Data preparation for gamma/hadron classification

Execise: split the data into train / test using sklearn

At this point we combine the two datasets into one big matrix and build a label vector $y$

In [ ]:
X = pd.concat([gammas[training_features], protons[training_features]])
y = np.concatenate([np.ones(len(gammas)), np.zeros(len(protons))])

In [ ]:
from sklearn.model_selection import train_test_split

X_test, X_train, y_test, y_train = train_test_split(X, y)

### Train the classifier

Now we can train any classifier we want on the prepared data.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

rf = DecisionTreeClassifier(max_depth=15, criterion='entropy')
rf.fit(X_train, y_train)

y_prediction = rf.predict(X_test)
y_prediction_proba = rf.predict_proba(X_test)

### Classifier Evaluation 

Check accuracy of the models and other metrics 

In [ ]:
importance = pd.Series(rf.feature_importances_, index=training_features)


plt.figure()
importance.sort_values().tail(20).plot.barh()
plt.tight_layout()

In [ ]:
from sklearn.metrics import accuracy_score, roc_curve, roc_auc_score

acc = accuracy_score(y_test, y_prediction)
auc = roc_auc_score(y_test, y_prediction_proba[:, 1])
fpr, tpr, thresholds = roc_curve(y_test, y_prediction_proba[:, 1])

In [ ]:

plt.figure()
plt.scatter(fpr, tpr, c=thresholds, vmax=1)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.gca().set_aspect(1)
plt.plot(fpr, tpr, '--', color='gray', alpha=0.5)
plt.text(0.5, 0.5, f'AuC ROC: {auc:0.03f} \nAccuracy: {acc:0.03f}')
plt.colorbar()
None

# 8. Redo, using cross-validation

Perform steps 3, 4, and 5 in one step using cross validation

In [ ]:
from sklearn.model_selection import cross_validate

rf = DecisionTreeClassifier(max_depth=12, criterion='entropy')

scoring = {'acc': 'accuracy',
           'auc': 'roc_auc',
           'recall': 'recall'}

results = cross_validate(rf, X, y, cv=5, scoring=scoring, return_train_score=True)
results

In [ ]:
auc = results['test_auc']
recall = results['test_recall']
acc = results['test_acc']

print(f'Area under RoC curve: {auc.mean():0.04f} ± {auc.std():0.04f}')
print(f'Accuracy:             {acc.mean():0.04f} ± {acc.std():0.04f}')
print(f'Recall:               {recall.mean():0.04f} ± {recall.std():0.04f}')

## 8. Physics

Now we could test our model and our hypothesis on real observed data. This part of the analysis is the most time 
consuming in general. It also requires more data than than this notebook can handle. 
After careful analysis one can produce an image of the gamma-ray sky

<img width="60%" src="https://www.mpi-hd.mpg.de/hfm/HESS/hgps/figures/HESS_J1813m126.png">

## Extra: Improving Classification


### Boosting and AdaBoost

Similar to the idea of combining many classifiers through bagging (like we did for the RandomForests) we now 
train many estimators in a sequential manner. In each iteration the data gets modified slightly using weights $w$
for each sample in the training data. In the first iteration the weights are all set to $w=1$

In each successive iteration the weights are updated. The samples that were incorrectly classified in the previous 
iteration get a higher weight. The weights for correctly classified samples get decreases. 
In other words: We increase the influence/importance of samples that are difficult to classify.

Predictions are performed by taking a weighted average of the single predictors.

The popular AdaBoost algorithms takes this a step further by optimizing the weight of each separate classifier 
in the ensemble.
The AdaBoost ensemble combines many learners in an iterative way. The learner at iteration $m$ is:

$$
 F_{m}(x)=F_{m-1}(x)+\gamma _{m}h_{m}(x)
$$

The choice of $F_0$ is problem specific.

Each weak learner produces a prediction $h(x_{m})$ for each sample in the training set. At each iteration $m$ a 
weak learner is fitted and assigned a coefficient $\gamma_{m}$ which is found by minimizing:

$$
\gamma_m = {\underset {\gamma }{\arg \min }} \sum_{i}^{N}E\bigl(F_{m-1}(x_{i})+\gamma h(x_{i})\bigr)
$$

where $E(F)$ is some error function and $x_i$ is the reweighted data sample.

In general this method can work with any classifying method. Traditionally it is being used with very small 
decision trees. 
The weights get used to select the split points during the minimization of the loss function in each node

$$
 \underset{(X, s) \in \, \mathbf{X} \times {S}}{\arg \max} IG(X,Y) =   \underset{(X, s) \in \, \mathbf{X} \times {S}}{\arg \max} ( H(Y) - H(Y |\, X) ).
$$

Below we try AdaBoost on the CTA data.


In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier(
    base_estimator=DecisionTreeClassifier(max_depth=2),
    n_estimators=100,
    learning_rate=0.5,
)
ada.fit(X_train, y_train)

y_prediction = ada.predict(X_test)
y_prediction_proba = ada.predict_proba(X_test)

In [ ]:
scores = np.array(list(ada.staged_score(X_test, y_test)))

plt.figure()
plt.plot(scores, '.')
plt.ylabel('Accuracy')
plt.xlabel('Iteration')
None

In [ ]:
acc = accuracy_score(y_test, y_prediction)
auc = roc_auc_score(y_test, y_prediction_proba[:, 1])
fpr, tpr, thresholds = roc_curve(y_test, y_prediction_proba[:, 1])

plt.figure()
plt.scatter(fpr, tpr, c=thresholds)
plt.plot(fpr, tpr, '--', color='gray', alpha=0.5)
plt.text(0.5, 0.5, f'AuC ROC: {auc:0.03f} \nAccuracy: {acc:0.03f}')
None

### Gradient Boosting 

Very similar to AdaBoost. Only this time we change the target label we train the classifiers for.

Formulate the general problem as follows (See Wikipedia):

Starts with a constant function $F_{0}(x)$ and some differentiable loss function $L$ and incrementally expands it in a greedy fashion:

$$
F_{0}(x)={\underset {\gamma }{\arg \min }}{\sum _{i=1}^{n}{L(y_{i},\gamma )}}
$$

$$
F_{m}(x)=F_{m-1}(x)+{\underset {h_{m}\in {\mathcal {H}}}{\operatorname {arg\,min} }}\left[{\sum _{i=1}^{n}{L(y_{i},F_{m-1}(x_{i})+h_{m}(x_{i}))}}\right]
$$

Finding the best $ h_{m}\in {\mathcal {H}}$ is computationally speaking impossible.
If we could find the perfect $h$ however, we know that 

$$
F_{m+1}(x_i)=F_{m}(x_i)+h(x_i)=y_i
$$

or, equivalently, 

$$
   h(x_i)= y_i - F_{m}(x_i)
$$

Note that for the mean squared error loss $\frac{1}{2}(y_i - F(x_i))^2$ this is equivalent to the negative 
gradient with respect to $F_i$.

For a general loss function we fit $h_{m}(x)$ to the residuals, or negative gradients 
$$
 r_{i, m}=-\left[{\frac {\partial L(y_{i},F(x_{i}))}{\partial F(x_{i})}}\right]_{F(x)=F_{m-1}(x)}\quad {\mbox{for }}i=1,\ldots ,n.
$$



Below we try it on CTA data again.


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

grb = GradientBoostingClassifier(
    verbose=True,
    n_estimators=300,
)
grb.fit(X_train, y_train)

y_prediction = grb.predict(X_test)
y_prediction_proba = grb.predict_proba(X_test)

In [ ]:
l = [accuracy_score(y_pred, y_test) for y_pred in grb.staged_predict(X_test)]

plt.figure()
plt.plot(range(len(l)), l, '.')
plt.ylabel('Accuracy')
plt.xlabel('Iteration')
None

In [ ]:
acc = accuracy_score(y_test, y_prediction)
auc = roc_auc_score(y_test, y_prediction_proba[:, 1])
fpr, tpr, thresholds = roc_curve(y_test, y_prediction_proba[:, 1])

plt.figure()
plt.scatter(fpr, tpr, c=thresholds)
plt.plot(fpr, tpr, '--', color='gray', alpha=0.5)
plt.text(0.5, 0.5, f'AuC ROC: {auc:0.03f} \nAccuracy: {acc:0.03f}')
None

More on gradient descent algorithms can be found in the Neural Network lecture.

Let's now test our all time favorite classifier. 

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=150,  max_depth=18, criterion='entropy')
rf.fit(X_train, y_train)

y_prediction = rf.predict(X_test)
y_prediction_proba = rf.predict_proba(X_test)

In [ ]:
acc = accuracy_score(y_test, y_prediction)
auc = roc_auc_score(y_test, y_prediction_proba[:, 1])
fpr, tpr, thresholds = roc_curve(y_test, y_prediction_proba[:, 1])

plt.figure()
plt.scatter(fpr, tpr, c=thresholds)
plt.plot(fpr, tpr, '--', color='gray', alpha=0.5)
plt.text(0.5, 0.5, f'AuC ROC: {auc:0.03f} \nAccuracy: {acc:0.03f}')
None